In [1]:
from jupyter_cadquery import show, open_viewer, set_defaults
import cadquery as cq
from build123d import *
cv = open_viewer("Build123d", cad_width=770, glass=True)
set_defaults(edge_accuracy=0.0001)

Overwriting auto display for cadquery Workplane and Shape


In [2]:
clock_radius = 10
with BuildSketch() as minute_indicator:
    with BuildLine() as outline:
        l1 = CenterArc((0, 0), clock_radius * 0.975, 0.75, 4.5)
        l2 = CenterArc((0, 0), clock_radius * 0.925, 0.75, 4.5)
        Line(l1 @ 0, l2 @ 0)
        Line(l1 @ 1, l2 @ 1)
    make_face()
    fillet(minute_indicator.vertices(), radius=clock_radius * 0.01)

with BuildSketch() as clock_face:
    Circle(clock_radius)
    with PolarLocations(0, 60):
        add(minute_indicator.sketch, mode=Mode.SUBTRACT)
    with PolarLocations(clock_radius * 0.875, 12):
        SlotOverall(clock_radius * 0.05, clock_radius * 0.025, mode=Mode.SUBTRACT)
    for hour in range(1, 13):
        with PolarLocations(clock_radius * 0.75, 1, -hour * 30 + 90, 360, rotate=False):
            Text(
                str(hour),
                font_size=clock_radius * 0.175,
                font_style=FontStyle.BOLD,
                mode=Mode.SUBTRACT,
            )

In [3]:
clock_face

In [4]:
densa = 7800/1e6 #carbon steel density g/mm^3
densb = 2700/1e6 #aluminum alloy
densc = 1020/1e6 #ABS
ms = Mode.SUBTRACT

In [5]:
with BuildPart() as p:
    with BuildSketch(Plane.XZ) as s2:
        SlotOverall((315-85-60)*2,160,rotation=90)
        split(bisect_by=Plane.XZ,keep=Keep.BOTTOM)
    extrude(amount=140/2,both=True)
    with BuildSketch(Plane.YZ) as s3:
        RectangleRounded(140,170*2,20,align=(Align.CENTER,Align.MIN))
    extrude(amount=160,both=True,mode=Mode.INTERSECT)
    with BuildSketch(Plane.YZ) as s4:
        with Locations((0,15)):
            RectangleRounded(140-30,170*2,5,align=(Align.CENTER,Align.MIN))
    extrude(amount=160,both=True,mode=ms)
    with BuildSketch(Plane.XZ) as s:
        SlotCenterToCenter(85*2,120,rotation=90)
        split(bisect_by=Plane.XZ)
    extrude(amount=80/2,both=True)
    with BuildSketch(Plane.XZ) as s5:
        with Locations((0,170/2)):
            Circle(60/2)
    extrude(amount=200,both=True,mode=ms)
    with BuildSketch(Plane.XZ) as s6:
        with Locations((0,-85)):
            Circle(60/2)
    extrude(amount=50,both=True)
    with BuildSketch(Plane.XZ) as s7:
        with Locations((0,-85)):
            Circle(40/2)
    extrude(amount=50,both=True,mode=ms)

In [6]:
p

In [7]:
with BuildPart() as p1:
    with BuildSketch() as sk1:
        RectangleRounded(75, 80, 4)
        with GridLocations(45, 50, 2, 2):
            Circle(4.5 / 2, mode=Mode.SUBTRACT)
    extrude(amount=-5)

    with BuildPart() as clamp_connector:
        with BuildSketch(Plane.YZ.offset(29 / 2)) as clamp_connector_sk:
            t = Trapezoid(60, 20, 45, 45, align=(Align.CENTER, Align.MIN))
        extrude(amount=4, both=True)
        mirror(about=Plane.YZ)

    with BuildPart() as bridge:
        with BuildSketch(Plane.YZ) as clamp_bridge_sk:
            top_edge: Edge = t.edges().sort_by(Axis.Y)[-1]
            bottom_edge: Edge = t.edges().sort_by(Axis.Y)[0]
            with BuildLine() as clamp_bridge_line:
                Polyline(
                    bottom_edge @ 0,
                    top_edge @ 1,
                    top_edge @ 0,
                    top_edge @ 0 - (0, 10),
                    top_edge @ 1 - (0, 10),
                    ((top_edge @ 1).X, (bottom_edge @ 0).Y),
                    close=True,
                )
            make_face()
        extrude(amount=29 / 2, both=True)

In [8]:
p1

In [9]:
with BuildPart() as p2:
    with BuildSketch() as s2:
        with Locations((80,0)):
            Circle(25)
        split(bisect_by=Plane.YZ.offset(78-14/2))
        with Locations((78,75/2)):
            Circle(14)
        mirror(about=Plane.XZ)
        make_hull()
    extrude(amount=15)
    with Locations((78,75/2,15),(78,-75/2,15)):
        CounterBoreHole(9/2,16/2,5)
with BuildPart() as p:
    with BuildSketch(Plane.XZ) as s:
        with BuildLine(Plane.XZ) as l:
            l1 = Line((0,0),(80+25,0))
            l2 = Line(l1@1,l1@1+(0,27))
            l3 = Line(l1@0,l1@0+(-25,0))
            l4 = Line(l3@1,l3@1+(0,70))
            l5 = Line(l4@1,l4@1+(50,0))
            l6 = PolarLine(l5@1,-32,-90+15,length_mode=LengthMode.VERTICAL)
            l7 = JernArc(l6@1,l6%1,15,90-15)
            l8 = Line(l7@1,l2@1)
        make_face()
    extrude(amount=25,both=True)
    with BuildSketch() as s3:
        with Locations((80/2,0)):
            SlotCenterToCenter(80,25*2)
    extrude(amount=70,mode=Mode.INTERSECT)
    add(p2.part,rotation=(0,0,.1)) #arbitrary rotation to fix errors
    with Locations(Plane.XY.offset(70)):
        CounterBoreHole(25/2,35/2,10)
    with BuildSketch(Plane.XZ) as s4:
        with BuildLine(Plane.XZ) as l2:
            m6 = PolarLine(l5@1,-32,-90+15,length_mode=LengthMode.VERTICAL)
            m7 = JernArc(m6@1,m6%1,15,90-15)
            m8 = Line(m6@0,m6@0+(7,0))
            m9 = PolarLine(m8@1,-7-6.765,-90+15,length_mode=LengthMode.VERTICAL)
            m10 = JernArc(m9@1,m6%1,30,90-15)
            m11 = Line(m10@1,(80+25-30,34))
            m12 = Line(m11@1,m11@1+(0,-7))
            m13 = Line(m12@1,m7@1)
        make_face()
    extrude(amount=3,both=True)
    with BuildSketch() as s5:
        Rectangle(50,6,align=(Align.MAX,Align.CENTER))
    extrude(amount=70,mode=ms)

In [10]:
p

In [11]:
with BuildPart() as p:
    with BuildSketch() as s:
        Circle(60)
        split(bisect_by=Plane.XZ,keep=Keep.BOTTOM)
        Rectangle(200-60,10,align=(Align.MIN,Align.MIN))
        make_hull(mode=Mode.REPLACE)
    extrude(amount=30)
    Cylinder(81/2,31*2,mode=Mode.SUBTRACT)
    with BuildSketch(Plane.YZ) as s2:
        with Locations((0,30/2)):
            SlotCenterToCenter(24*2,12)
    extrude(amount=200,both=True,mode=ms)
    with Locations((70,13)):
        Cylinder(18/2,30*2,mode=ms)
    with PolarLocations(50,3,45):
        Cylinder(10/2,30*2,mode=ms)
    split(bisect_by=Plane.XY.rotated((6,0,0)))
    split(bisect_by=Plane.XY.offset(30).rotated((-6,0,0)),keep=Keep.BOTTOM)

In [12]:
p

In [13]:
with BuildPart() as p:
    with BuildSketch() as s:
        Circle(60)
        split(bisect_by=Plane.XZ,keep=Keep.BOTTOM)
        Rectangle(200-60,10,align=(Align.MIN,Align.MIN))
        make_hull(mode=Mode.REPLACE)
    extrude(amount=30)
    Cylinder(81/2,31*2,mode=Mode.SUBTRACT)
    with BuildSketch(Plane.YZ) as s2:
        with Locations((0,30/2)):
            SlotCenterToCenter(24*2,12)
    extrude(amount=200,both=True,mode=ms)
    with Locations((70,13)):
        Cylinder(18/2,30*2,mode=ms)
    with PolarLocations(50,3,45):
        Cylinder(10/2,30*2,mode=ms)
    split(bisect_by=Plane.XY.rotated((6,0,0)))
    split(bisect_by=Plane.XY.offset(30).rotated((-6,0,0)),keep=Keep.BOTTOM)      